
<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/14_Forward_Mode_AD.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🥋 Lekcja 14: Forward Mode AD (Przyszłość różniczkowania)

Klasyczne uczenie maszynowe (Backpropagation) to **Reverse Mode**.
*   Świetne, gdy: `Inputs >> Outputs` (np. Sieć neuronowa: milion wag -> 1 Loss).
*   Wada: Musi trzymać cały graf w pamięci.

**Forward Mode AD:**
*   Świetne, gdy: `Outputs >> Inputs` (np. Generative AI, Fizyka, Jacobiany).
*   Zaleta: Liczy pochodną "w locie" (w trakcie forward pass). Nie zajmuje pamięci na graf!

**Matematyka (Liczby Dualne):**
Zamiast liczby $x$, wprowadzamy parę $(x, \dot{x})$, gdzie $\dot{x}$ to "ziarno" gradientu (tangent).
Każda operacja liczy wynik i od razu jego pochodną:
$$ f(x, \dot{x}) = (f(x), f'(x) \cdot \dot{x}) $$

In [1]:
import torch
import torch.autograd.forward_ad as fwAD # Nowy moduł w PyTorch

# Funkcja wektorowa: 1 wejście -> 3 wyjścia
# f(x) = [x, x^2, x^3]
def func(x):
    return torch.stack([x, x**2, x**3])

x = torch.tensor([3.0])

print(f"Wejście: {x}")
print(f"Wyjście: {func(x)}")

Wejście: tensor([3.])
Wyjście: tensor([[ 3.],
        [ 9.],
        [27.]])


## Podejście 1: Klasyczne (Reverse Mode)

Gdybyśmy chcieli policzyć pochodną `func` względem `x` tradycyjnie, musielibyśmy zrobić `backward` dla każdego elementu wyjścia osobno (lub użyć `jacobian`).

Dla funkcji $f: \mathbb{R}^1 \to \mathbb{R}^{1000}$, Reverse Mode jest 1000x wolniejszy niż Forward Mode!

In [2]:
# Klasyczne podejście (wymaga requires_grad)
x_rev = torch.tensor([3.0], requires_grad=True)
y_rev = func(x_rev)

# Musimy liczyć gradienty dla każdego wyjścia osobno?
# Albo użyć sztuczki z sumą, albo obliczyć Jakobian.
# Zobaczmy Jakobian (macierz pochodnych)
J_rev = torch.autograd.functional.jacobian(func, x_rev)

print("--- REVERSE MODE JACOBIAN ---")
print(J_rev.squeeze())
# Oczekujemy: [1, 2x, 3x^2] -> dla x=3: [1, 6, 27]

--- REVERSE MODE JACOBIAN ---
tensor([ 1.,  6., 27.])


## Podejście 2: Forward Mode AD

Używamy `fwAD`.
1.  Otwieramy kontekst `dual_level`.
2.  Tworzymy **Liczbę Dualną** (`make_dual`): pakujemy wartość $x$ i styczną $\dot{x}$ (tangent). Zazwyczaj $\dot{x}=1$.
3.  Uruchamiamy funkcję raz.
4.  Rozpakowujemy wynik (`unpack_dual`). Dostajemy od razu wynik i gradient!

Zero budowania grafu wstecznego.

In [3]:
# Krok 1: Kontekst
with fwAD.dual_level():
    
    # Krok 2: Tworzymy Dual Tensor (Wartość + Tangent)
    # Tangent = 1.0 oznacza, że liczymy pochodną df/dx
    dual_input = fwAD.make_dual(x, torch.ones_like(x))
    
    # Krok 3: Forward (Obliczenia lecą normalnie)
    dual_output = func(dual_input)
    
    # Krok 4: Rozpakowanie
    result, jvp = fwAD.unpack_dual(dual_output)

print("--- FORWARD MODE RESULT ---")
print(f"Wynik funkcji: {result}")
print(f"Gradient (JVP): {jvp}") # Jacobian-Vector Product

--- FORWARD MODE RESULT ---
Wynik funkcji: tensor([[ 3.],
        [ 9.],
        [27.]])
Gradient (JVP): tensor([[ 1.],
        [ 6.],
        [27.]])


## Benchmark: Reverse vs Forward

Zróbmy ekstremalny przykład.
Funkcja bierze 1 liczbę i zwraca 10 000 liczb.
$$ f(x) = [x^0, x^1, ..., x^{9999}] $$

*   **Reverse Mode:** Musi przejść wstecz przez graf 10 000 razy (dla każdego wyjścia), żeby zbudować pełny Jakobian.
*   **Forward Mode:** Przechodzi raz w przód i niesie gradient ze sobą.

In [4]:
import time

# Funkcja 1 -> 10000
def massive_expansion(x):
    # Tworzymy potęgi od 0 do 9999
    powers = torch.arange(10000, device=x.device)
    return x ** powers

x_bench = torch.tensor([1.0001], requires_grad=True) # Mała liczba > 1

# 1. Reverse Mode (Jacobian)
start = time.time()
# jacobian w PyTorch używa reverse mode domyślnie
J_rev = torch.autograd.functional.jacobian(massive_expansion, x_bench)
print(f"Reverse Mode Time: {time.time() - start:.4f} s")

# 2. Forward Mode
start = time.time()
with fwAD.dual_level():
    dual_x = fwAD.make_dual(x_bench, torch.ones_like(x_bench))
    dual_y = massive_expansion(dual_x)
    _, J_fwd = fwAD.unpack_dual(dual_y)
print(f"Forward Mode Time: {time.time() - start:.4f} s")

# Sprawdźmy poprawność
print(f"\nCzy wyniki są takie same? {torch.allclose(J_rev.squeeze(), J_fwd.squeeze())}")

Reverse Mode Time: 1.3660 s
Forward Mode Time: 0.7635 s

Czy wyniki są takie same? True


## 🥋 Black Belt Summary

1.  **Reverse Mode (`backward`)**: Używaj, gdy trenujesz sieci neuronowe (Loss to jedna liczba, a wag są miliony). Koszt zależy od liczby wejść.
2.  **Forward Mode (`fwAD`)**: Używaj, gdy liczysz Jakobian funkcji, która ma mało wejść, a dużo wyjść (lub gdy output jest wektorem). Koszt zależy od liczby wejść (w naszym teście 1 wejście = super szybko).

**Gdzie to spotkasz?**
*   **PINN (Physics-Informed Neural Networks):** Rozwiązywanie równań różniczkowych.
*   **Optymalizacja drugiego rzędu:** Liczenie Hessianu (jako Jacobiana z Gradientu) można przyspieszyć, mieszając Reverse i Forward mode.